In [1]:
# Call the relevant python packages
import numpy as np
import pandas as pd
import glob, os                          # Operating system module and pathnames

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import scipy.constants as const
import scipy.stats as stats

from astropy import units as u
from astropy.io import ascii, fits
from astropy.table import join, hstack, Table, Column, vstack
from astropy.coordinates import Angle, Latitude, Longitude, SkyCoord

#import SED Fitter module
from sedfitter import fit, plot, plot_params_1d, plot_params_2d
from sedfitter import filter_output, write_parameters, write_parameter_ranges
from sedfitter.extinction import Extinction

# Set colour palette & plot layouts
import seaborn as sns     #; sns.set()
sns.set_context("paper")   # "paper", "notebook", "talk", "poster"

# Output plots in Notebook
#%matplotlib notebook
%matplotlib inline

In [10]:
# Read in the parameter and parameter range files - both are required as inputs
param_file = '/scratch/s1625982/NGC6822/env/Output_SED/YSO_2006/params1to100.csv'
p_data = ascii.read(param_file)
param_range_file = '/scratch/s1625982/NGC6822/env/Output_SED/YSO_2006/param_ranges1to100.csv'
pr_data = ascii.read(param_range_file)

# Define solar radius, Stefan-Boltzmann constant and temperature of the Sun
sigma = const.Stefan_Boltzmann
R_sun = 6.957e+08 * u.m
T_sun = 5778 * u.Kelvin

# Define function to write a flag to a column at indexed positions.
# This writes the flags to the parameter file (not the ranges file)
def write_column(crit,name,flag):
    print("Number of stars in cut "+name+' '+str(flag)+": {}".format(len(crit[0])))
    for i in range(len(crit[0])):
        p_data[name][crit[0][i]]=flag

# Create columns for luminosity in both files and calculate in units of L☉
p_data['luminosity']=0
p_data['luminosity']=(((p_data['rstar'])**2)*((p_data['tstar'])**4)/T_sun**4)
pr_data['lum_best']=0
pr_data['lum_best']=(((pr_data['rstar_best'])**2)*((pr_data['tstar_best'])**4)/T_sun**4)

# Determine YSO evolutionary stages
p_data['stage']=0
idx_stage_1 = np.where(p_data['mdot']>=10**-6)
idx_stage_2 = np.where((p_data['mdot']<10**-6) & (p_data['mdisk']>10**-6))
idx_stage_3 = np.where((p_data['mdot']<10**-6) & (p_data['mdisk']<10**-6))
write_column(idx_stage_1,'stage',1)
write_column(idx_stage_2,'stage',2)
write_column(idx_stage_3,'stage',3)

# Print the total number of fits in the parameter file
num_fits = len(p_data['source_name'])
print("The number of fits in this file is",num_fits)
# Print the total number of sources in the ranges file
num_sources = len(pr_data['source_name'])
print("The number of sources in this file is",num_sources)

# Build array of n_fits corresponding to each source.
# This writes a '1' to a new column to flag if that row starts a new source
source_num = 0
p_data['new_source?']=0
arr_new_source = np.array([])
for i in range(num_fits):
    new_source = p_data['source_name'][i] - source_num
    if new_source == 1:
        arr_new_source = np.append(arr_new_source,np.array([i]))
        p_data['new_source?'][i]=1
    source_num = p_data['source_name'][i]

# Define a list of parameters to find MADs for
param_list = ['chi2','av','scale','time','massc','rstar','tstar','mdot',
             'rmax','theta','rmine','mdisk','rmaxd','rmind','rmind(au)',
             'rc','rchole','zmin','a','b','alpha','rhoconst','rhoamb',
             'mdotdisk','incl.','av_int','tau60','ltot','h100']
    
# Find MAD for each parameter above
# Set up empty columns for each MAD in ranges file
p_data['L_MAD']=0
pr_data['L_MAD']=0
for i in range(len(param_list)):
    param_mad_name = param_list[i]+'_MAD'
    p_data[param_mad_name]=0
    pr_data[param_mad_name]=0
    
# For each source in the table find the MADs
idx_new_source = np.where(p_data['new_source?']==1)

# Cycle through each fit for each source to get luminosity MAD
counter = 0
for i in range(num_sources):
    counter = counter + 1
    L_best = pr_data['lum_best'][i]
    n_fits_for_source = pr_data['n_fits'][i]
    arr_MAD = np.array([])
    if n_fits_for_source > 1:
        for j in range(n_fits_for_source-1):
            dev = np.array([abs(L_best - p_data['luminosity'][counter])])
            arr_MAD = np.append(arr_MAD, dev)
            counter = counter + 1
        MAD_value = np.median(arr_MAD)
    else:
        MAD_value = 0

    pr_data['L_MAD'][i] = MAD_value
    
# Run this again for all other non-user-calculated parameters
# Cycle through all parameters listed above
for n in range(len(param_list)):
    counter = 0
    for i in range(num_sources):
        counter = counter + 1
        param_best_name = param_list[n]+'_best'
        param_best = pr_data[param_best_name][i]
        n_fits_for_source = pr_data['n_fits'][i]
        arr_MAD = np.array([])
        if n_fits_for_source > 1:
            for j in range(n_fits_for_source-1):
                dev = np.array([abs(param_best - p_data[param_list[n]][counter])])
                arr_MAD = np.append(arr_MAD, dev)
                counter = counter + 1
            MAD_value = np.median(arr_MAD)
        else:
            MAD_value = 0

        param_mad_name = param_list[n]+'_MAD'
        pr_data[param_mad_name][i] = MAD_value
    
pr_data.show_in_notebook()

Number of stars in cut stage 1: 436
Number of stars in cut stage 2: 199
Number of stars in cut stage 3: 256
The number of fits in this file is 891
The number of sources in this file is 100


source_name,n_data,n_fits,chi2_min,chi2_best,chi2_max,av_min,av_best,av_max,scale_min,scale_best,scale_max,time_max,time_best,time_min,massc_max,massc_best,massc_min,rstar_max,rstar_best,rstar_min,tstar_max,tstar_best,tstar_min,mdot_max,mdot_best,mdot_min,rmax_max,rmax_best,rmax_min,theta_max,theta_best,theta_min,rmine_max,rmine_best,rmine_min,mdisk_max,mdisk_best,mdisk_min,rmaxd_max,rmaxd_best,rmaxd_min,rmind_max,rmind_best,rmind_min,rmind(au)_max,rmind(au)_best,rmind(au)_min,rc_max,rc_best,rc_min,rchole_max,rchole_best,rchole_min,zmin_max,zmin_best,zmin_min,a_max,a_best,a_min,b_max,b_best,b_min,alpha_max,alpha_best,alpha_min,rhoconst_max,rhoconst_best,rhoconst_min,rhoamb_max,rhoamb_best,rhoamb_min,mdotdisk_max,mdotdisk_best,mdotdisk_min,incl._max,incl._best,incl._min,av_int_max,av_int_best,av_int_min,tau60_max,tau60_best,tau60_min,ltot_max,ltot_best,ltot_min,h100_max,h100_best,h100_min,lum_best,L_MAD,chi2_MAD,av_MAD,scale_MAD,time_MAD,massc_MAD,rstar_MAD,tstar_MAD,mdot_MAD,rmax_MAD,theta_MAD,rmine_MAD,mdisk_MAD,rmaxd_MAD,rmind_MAD,rmind(au)_MAD,rc_MAD,rchole_MAD,zmin_MAD,a_MAD,b_MAD,alpha_MAD,rhoconst_MAD,rhoamb_MAD,mdotdisk_MAD,incl._MAD,av_int_MAD,tau60_MAD,ltot_MAD,h100_MAD
1,8,3,182.0,182.0,193.0,0.0,0.0,0.0,2.65,2.65,2.65,178000.0,178000.0,233000.0,10.0,10.0,14.1,3.88,3.88,4.71,26000.0,26000.0,30500.0,1.45e-05,1.45e-05,1.53e-05,100000.0,100000.0,100000.0,18.0,19.7,19.7,1.0,1.0,1.0,0.336,0.379,0.379,81.8,167.0,167.0,1.0,1.0,1.0,10.8,11.2,11.2,81.8,167.0,167.0,81.8,167.0,167.0,0.736,0.995,0.995,2.01,2.01,2.06,1.01,1.01,1.06,0.0042,0.0974,0.0974,5.8e-21,8.74e-21,8.74e-21,5.54e-21,5.54e-21,5.62e-21,6.15e-06,0.000192,0.000192,81.4,87.1,87.1,25600.0,51200.0,103000.0,4.35,4.35,4.56,17800.0,18600.0,18600.0,5.92,6.49,6.49,6172,5525,10,0,0,27500,2,0,2250,0,0,0,0,0,42,0,0,42,42,0,0,0,0,0,0,0,2,38700,0,400,0
2,5,1,357.0,357.0,357.0,5.33,5.33,5.33,2.67,2.67,2.67,497000.0,497000.0,497000.0,18.9,18.9,18.9,5.58,5.58,5.58,34500.0,34500.0,34500.0,7.25e-06,7.25e-06,7.25e-06,100000.0,100000.0,100000.0,25.1,25.1,25.1,1.0,1.0,1.0,0.402,0.402,0.402,1950.0,1950.0,1950.0,1.0,1.0,1.0,16.5,16.5,16.5,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0,0.969,0.969,0.969,2.01,2.01,2.01,1.01,1.01,1.01,0.0235,0.0235,0.0235,4.73e-21,4.73e-21,4.73e-21,4.73e-21,4.73e-21,4.73e-21,6.09e-06,6.09e-06,6.09e-06,87.1,87.1,87.1,7810.0,7810.0,7810.0,1.86,1.86,1.86,40100.0,40100.0,40100.0,7.36,7.36,7.36,39576,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,3,545.0,545.0,556.0,0.0,0.215,9.74,2.66,2.66,2.72,258000.0,447000.0,447000.0,20.0,20.0,21.9,5.76,5.76,6.03,35300.0,35300.0,36600.0,8.06e-06,1.55e-05,2.91e-05,100000.0,100000.0,100000.0,24.8,26.5,29.5,1.0,1.0,1.0,0.00143,0.00304,0.262,77.3,77.3,208.0,1.0,1.0,1.0,17.8,17.8,20.1,77.3,77.3,208.0,77.3,77.3,208.0,0.698,0.874,0.874,2.01,2.03,2.05,1.01,1.03,1.05,0.0017,0.071,0.071,4.74e-21,4.74e-21,7.29e-21,4.43e-21,4.51e-21,7.29e-21,5.11e-09,2.71e-06,3.59e-06,87.1,87.1,87.1,653.0,2310.0,28100.0,2.22,2.22,2.66,46600.0,46600.0,58800.0,5.47,7.03,7.03,46220,7017,9,4,0,161000,1,0,750,0,0,2,0,0,90,0,1,90,90,0,0,0,0,0,0,0,0,13723,0,6800,1
4,4,3,944.0,944.0,952.0,8.21,15.4,15.4,2.7,2.7,2.72,1530.0,1530.0,1530.0,47.5,47.5,47.5,741.0,741.0,741.0,5940.0,5940.0,5940.0,0.001,0.001,0.001,100000.0,100000.0,100000.0,1.32,1.32,1.32,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.85,8.85,8.85,8.85,8.85,8.85,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.27e-20,6.27e-20,6.27e-20,1.48e-20,1.48e-20,1.48e-20,-1.0,-1.0,-1.0,41.4,41.4,75.5,709.0,709.0,781.0,690.0,690.0,690.0,613000.0,613000.0,613000.0,-1.0,-1.0,-1.0,613298,0,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,66,0,0,0
5,5,12,221.0,221.0,231.0,13.9,40.0,40.0,2.65,2.66,2.68,1490000.0,1490000.0,1900000.0,14.5,17.4,17.4,4.82,5.34,5.34,30900.0,33400.0,33400.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.96e-07,9.96e-07,0.285,5970.0,8460.0,8460.0,1.0,1.92,3.49,11.5,28.1,39.1,5970.0,8570.0,8570.0,5970.0,8570.0,8570.0,0.518,0.8

In [3]:
p_data.show_in_notebook()

source_name,n_data,n_fits,fit_id,model_name,chi2,av,scale,time,massc,rstar,tstar,mdot,rmax,theta,rmine,mdisk,rmaxd,rmind,rmind(au),rc,rchole,zmin,a,b,alpha,rhoconst,rhoamb,mdotdisk,incl.,av_int,tau60,ltot,h100,luminosity,stage,new_source?,L_MAD
1,8,3,1,3005847_1,181.531,0.0,2.653,178000.0,10.0,3.88,26000.0,1.45e-05,100000.0,19.7,1.0,0.379,167.0,1.0,11.2,167.0,167.0,0.995,2.01,1.01,0.0974,8.74e-21,5.54e-21,0.000192,87.1,51200.0,4.35,18600.0,6.49,6172,1,1,0
1,8,3,2,3005847_2,191.655,0.0,2.653,178000.0,10.0,3.88,26000.0,1.45e-05,100000.0,19.7,1.0,0.379,167.0,1.0,11.2,167.0,167.0,0.995,2.01,1.01,0.0974,8.74e-21,5.54e-21,0.000192,81.4,25600.0,4.35,18600.0,6.49,6172,1,0,0
1,8,3,3,3012163_1,193.413,0.0,2.653,233000.0,14.1,4.71,30500.0,1.53e-05,100000.0,18.0,1.0,0.336,81.8,1.0,10.8,81.8,81.8,0.736,2.06,1.06,0.0042,5.8e-21,5.62e-21,6.15e-06,87.1,103000.0,4.56,17800.0,5.92,17223,1,0,0
2,5,1,1,3013166_1,356.701,5.329,2.674,497000.0,18.9,5.58,34500.0,7.25e-06,100000.0,25.1,1.0,0.402,1950.0,1.0,16.5,1950.0,1950.0,0.969,2.01,1.01,0.0235,4.73e-21,4.73e-21,6.09e-06,87.1,7810.0,1.86,40100.0,7.36,39576,1,1,0
3,5,3,1,3017183_1,545.187,0.215,2.663,447000.0,20.0,5.76,35300.0,1.55e-05,100000.0,26.5,1.0,0.00304,77.3,1.0,17.8,77.3,77.3,0.874,2.03,1.03,0.071,4.74e-21,4.51e-21,2.71e-06,87.1,2310.0,2.22,46600.0,7.03,46220,1,1,0
3,5,3,2,3002844_1,553.237,9.741,2.724,314000.0,21.9,6.03,36600.0,8.06e-06,100000.0,24.8,1.0,0.262,128.0,1.0,20.1,128.0,128.0,0.698,2.01,1.01,0.0022,7.29e-21,7.29e-21,3.59e-06,87.1,28100.0,2.66,58800.0,5.47,58539,1,0,0
3,5,3,3,3015747_1,556.279,0.0,2.704,258000.0,20.3,5.8,35500.0,2.91e-05,100000.0,29.5,1.0,0.00143,208.0,1.0,18.1,208.0,208.0,0.719,2.05,1.05,0.0017,6.2e-21,4.43e-21,5.11e-09,87.1,653.0,2.66,48000.0,5.97,47935,1,0,0
4,4,3,1,3008772_8,943.802,15.42,2.704,1530.0,47.5,741.0,5940.0,0.001,100000.0,1.32,1.0,0.0,-1.0,-1.0,-1.0,8.85,8.85,-1.0,-1.0,-1.0,-1.0,6.27e-20,1.48e-20,-1.0,41.4,709.0,690.0,613000.0,-1.0,613298,1,1,0
4,4,3,2,3008772_4,951.7,10.93,2.714,1530.0,47.5,741.0,5940.0,0.001,100000.0,1.32,1.0,0.0,-1.0,-1.0,-1.0,8.85,8.85,-1.0,-1.0,-1.0,-1.0,6.27e-20,1.48e-20,-1.0,69.5,770.0,690.0,613000.0,-1.0,613298,1,0,0
4,4,3,3,3008772_3,952.195,8.213,2.724,1530.0,47.5,741.0,5940.0,0.001,100000.0,1.32,1.0,0.0,-1.0,-1.0,-1.0,8.85,8.85,-1.0,-1.0,-1.0,-1.0,6.27e-20,1.48e-20,-1.0,75.5,781.0,690.0,613000.0,-1.0,613298,1,0,0
